# **기말고사 내용 총 정리**

In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import gymnasium as gym
#---#
import collections
import random

from matplotlib.animation import FuncAnimation
import IPython

## **1. 유용한 기능**

---

### **A. 옵티마이저 고급**

`-` 텐서를 그대로 파라미터로서 사용하는 법

* `iterable`이기만 하면 됨

In [ ]:
lamb = torch.tensor(1.0, requires_grad = True)
optimize = torch.optim.SGD([lamb], lr = 0.1)

`-` 시퀸스 없이 파라미터 삽입

In [ ]:
linr1 = torch.nn.Linear(1, 32)
relu = torch.nn.ReLU()
linr2 = torch.nn.Linear(32, 1)

params = list(linr1.parameters()) + list(linr2.parameters())
optimizr = torch.optim.Adam(params)

### **B. gymnasium**

## **2. 추천 시스템**

---

`-` 데이터 전처리

In [ ]:
x1 = torch.tensor(df.x1.map({v:i for i, v in enumerate(set(df.x1))}))
x2 = torch.tensor(df.x2.map({v:i for i, v in enumerate(set(df.x2))}))
y = torch.tensor(df.y).reshape(-1, 1).float()

X = torch.stack([x1, x2], axis = 1)

### **A. MF-based 추천 시스템**

`-` 클래스 설계

In [ ]:
class MF_recommend(torch.nn.Module) :
    def __init__(self) :
        super().__init__()

        self.ebdd1 = torch.nn.Embedding(9, 2)
        self.ebdd2 = torch.nn.Embedding(8, 2)
        self.bias1 = torch.nn.Embedding(9, 1)
        self.bias2 = torch.nn.Embedding(8, 1)
        self.sig = torch.nn.Sigmoid()

    def forward(self, X) :
        x1 = X[:, 0]
        x2 = X[:, 1]

        yhat = self.sig((self.ebdd1(x1) * self.ebdd2(x2)).sum(axis = 1).reshape(-1, 1) + self.bias1(x1) + self.bias2(x2))*5
        return yhat

`-` 학습

In [ ]:
net = MF_recommend()

loss_fn = torch.nn.MSELoss()
optimizr = torch.optim.Adam(net.parameters())

#---#
for epoc in range(3000) :
    yhat = net(X)
    loss = loss_fn(yhat, y)
    loss.backward()
    optimizr.step()
    optimizr.zero_grad()

`-` 예측

In [ ]:
xx1 = torch.tensor([0, 1, 8])
xx2 = torch.tensor([0, 7, 2])
XX = torch.stack([xx1, xx2], axis = 1)

net(XX)

### **B. NN-based 추천 시스템**

`-` 클래스 선언

In [ ]:
class NNbased(torch.nn.Module):
    def __init__(self):
        super().__init__()
        #--#
        self.ebdd1 = torch.nn.Embedding(9,2)
        self.ebdd2 = torch.nn.Embedding(8,2)
        self.b1 = torch.nn.Embedding(9,1)
        self.b2 = torch.nn.Embedding(8,1)
        self.sig = torch.nn.Sigmoid()
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(6,15),
            torch.nn.ReLU(),
            torch.nn.Linear(15,1),
            torch.nn.Sigmoid()
        )
        
    def forward(self, X):
        x1 = X[:, 0]
        x2 = X[:, 1]
        
        W_feature = self.ebdd1(x1)
        W_bias = self.b1(x1)
        M_feature = self.ebdd2(x2)
        M_bias = self.b2(x2)
        
        Z = torch.concat([W_feature, M_feature, W_bias, M_bias],axis=1)
        
        yhat = self.mlp(Z) * 5 
        
        return yhat

`-` 학습

In [ ]:
net = NNbased()
loss_fn = torch.nn.MSELoss()
optimizr = torch.optim.Adam(net.parameters(),lr=0.1)
#--# 
for epoc in range(3000):
    yhat = net(X)
    
    loss = loss_fn(yhat,y)
    loss.backward()
    
    optimizr.step()
    optimizr.zero_grad()

`-` 예측

In [ ]:
xx1 = torch.tensor([0, 1, 8])
xx2 = torch.tensor([0, 7, 2])
XX = torch.stack([xx1, xx2], axis = 1)

net(XX)

## **3. 순환 신경망**

### **A. hidden feature**

`-` $h$는 사실 문자열을 숫자로 바꾼 표현으로 해석할 수 있음. 즉, 원-핫 인코딩과 다른 형태의 숫자표현으로 해석 가능.

* `hidden feature`로서 작용함 -> 학습시켜 생산된 피쳐

`-` 사실 $h$는 원-핫 인코딩보다 약간 더 (1) 액기스만 남은 느낌 + (2) 숙성된 느낌을 준다.

* (why 1) $h$는 $x$보다 $y$를 예측함에 좀 더 직접적인 역할을 한다. 즉, $x$ 숫자보다 $h$ 숫자가 잘 정리되어 있고(차원이 낮고), 입력의 특징을 잘 정리한 의미있는 숫자이다.
* (why 2) $x$는 학습없이 그냥 얻어지는 숫자표현이지만, $h$는 학습을 통하여 고치고 고치고 고치고 고친 숫자표현이다.

### **B. 하이퍼볼릭 탄젠트 시그모이드**

`-` `torch.nn.Tanh()`

* 실수를 `(-1, 1)`의 값으로 변환
* 기존 로지스틱 시그모이드의 경우 그 값이 0에 가까워질 수 있어 `weight`에 둔감할 수 있음 -> 단점 해소
* 여러 개의 은닉 노드를 사용하여 변환을 더욱 다양화할 수 있음

### **C. RNN 차원 표기**

* X.shape == $(L, H_{\text{in}})$ : 문자 길이 / 고유 문자열 수
* h.shape == $(L, H_{\text{out}})$
* y.shape == $(L, Q)$ : 문자 길이 / 고유 문자열 수
* Xt.shape == $(H_{\text{in}},)$
* ht.shape == $(H_{\text{out}},)$
* yt.shape == $(Q,)$

### **D. rNNCell class 생성**

`-` 클래스 선언

In [ ]:
class rNNCell(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.i2h = torch.nn.Linear(4,2)
        self.h2h = torch.nn.Linear(2,2)
        self.tanh = torch.nn.Tanh()
        
    def forward(self,Xt,ht): # 엄밀하게는 h_{t-1}
        ht = self.tanh(self.i2h(Xt) + self.h2h(ht)) 
        
        return ht

`-` 학습

In [ ]:
rnncell = rNNCell()
cook = torch.nn.Linear(2,4)
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(
    list(rnncell.parameters()) + 
    list(cook.parameters()),
    lr=0.1
)

#---#
L = len(X)
for epoc in range(200):
    # 1~2 
    loss = 0
    ht = torch.zeros(2) # 첫 ht는 맹물
    for t in range(L):
        Xt, yt = X[t], y[t]
        ht = rnncell(Xt,ht)
        ot = cook(ht)
        loss = loss_fn(ot,yt) + loss 
    loss = loss / L 
    # 3
    loss.backward()
    # 4 
    optimizr.step()
    optimizr.zero_grad()

`-` 예측

In [ ]:
h = torch.zeros(L,2) ## 공간 생성
h0 = torch.zeros(2) ## 초기값 : h0
h[0] = rnncell(X[0], h0) ## h1

for t in range(1,L):
    h[t] = rnncell(X[t], h[t-1])

yhat = torch.nn.functional.softmax(cook(h),dim=1) 

### **E. RNNCell 모듈 사용**

* 동일한 방식으로 `rNNCell` 클래스를 설계했기 때문에 사용법은 같음

`-` 학습

In [ ]:
rnncell = torch.nn.RNNCell(
    input_size = 4, #X.shape = (L,4)
    hidden_size= 2, # h.shape = (L,2) 
)
cook = torch.nn.Linear(2,4)
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(
    list(rnncell.parameters()) + 
    list(cook.parameters()),
    lr=0.1
)
#---#
L = len(X)
for epoc in range(200):
    # 1~2 
    loss = 0
    ht = torch.zeros(2) # 첫 ht는 맹물
    for t in range(L):
        Xt, yt = X[t], y[t]
        ht = rnncell(Xt,ht)
        ot = cook(ht)
        loss = loss_fn(ot,yt) + loss 
    loss = loss / L 
    # 3
    loss.backward()
    # 4 
    optimizr.step()
    optimizr.zero_grad()

`-` 예측

In [ ]:
h = torch.zeros(L,2) ## 공간 생성
h0 = torch.zeros(2) ## 초기값 : h0
h[0] = rnncell(X[0], h0) ## h1

for t in range(1,L):
    h[t] = rnncell(X[t], h[t-1])

yhat = torch.nn.functional.softmax(cook(h),dim=1) 

`-` 파라미터 추출

In [ ]:
torch.manual_seed(42)
rnncell = torch.nn.RNNCell(4,2)
rnncell.weight_ih.data = _rnncell.i2h.weight.data
rnncell.weight_hh.data =  _rnncell.h2h.weight.data
rnncell.bias_ih.data = _rnncell.i2h.bias.data
rnncell.bias_hh.data = _rnncell.h2h.bias.data

### **F. RNN 모듈**

`-` 학습

In [ ]:
rnn = torch.nn.RNN(
    input_size= 4, # X.shape=(L,H_in)
    hidden_size = 2, # h.shape=(L,H_out)
    num_layers = 1, # 어느 방향에서 레이어가 들어오는지의 차원. 피쳐가 몇개인지 느낌? 디폴트는 1
    bidirectional = False # bi-directional : True이면 뒤에서부터도 읽을 수 있는 양방향 네트워크. 디폴트는 False
)
cook = torch.nn.Linear(2,4)
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(list(rnn.parameters())+list(cook.parameters()),lr=0.1)

#---#
for epoc in range(200) :
    ## 초기화 하는 경우
    # h0 = torch.zeros(1, 2) ## D*num_layers == 1*1
    # h, hL = rnn(X, h0) ## (h, h[L])
    h, hL = rnn(X) ## 초기화 안하고 그냥 넣어도 돌아감
    netout = cook(h)
    
    loss = loss_fn(netout, y)
    loss.backward()
    optimizr.step()
    optimizr.zero_grad()

`-` 예측

In [ ]:
h, hL = rnn(X)
yhat = torch.nn.functional.softmax(cook(h),dim=1)

## **4. 강화 학습**

> 마음에 들지 않지만 꼭 외워야 하는것
>
> 1.  `env.step`은 항상 next_state, reward, terminated, truncated, info
>     를 리턴한다. – 짐나지엄 라이브러리 규격때문
> 2.  `env.reset`은 환경을 초기화할 뿐만 아니라, state, info를 반환하는
>     기능도 있다. – 짐나지엄 라이브러리 규격때문
> 3.  `player`는 항상 `state`와 `next_state`를 구분해서 저장한다.
>     (다른변수들은 그렇지 않음) 이는 강화학습이
>     MDP(마코프체인+행동+보상)구조를 따르기 때문에 생기는 고유한
>     특징이다 – 이론적인 이유

### **A. `q_table`**

`-` **Bandit Game**

* 클래스 선언

In [1]:
class Bandit :
    def __init__(self) :
        self.reward = None

    def step(self, action) :
        if action == 0 :
            self.reward = 1
        else :
            self.reward = 10

        return self.reward


class Agent :
    def __init__(self) :
        self.n_experience = 0

        self.action = None
        self.reward = None
        self.actions = collections.deque(maxlen = 500)
        self.rewards = collections.deque(maxlen = 500)

        self.action_space = [0, 1]
        self.q_table = None

    def act(self) :
        if self.n_experience < 20 :
            self.action = np.random.choice(self.action_space)
        else :
            self.action = self.q_table.argmax()

    def save_experience(self) :
        self.actions.append(self.action)
        self.rewards.append(self.reward)
        self.n_experience += 1

    def learn(self) :
        actions = np.array(self.actions)
        rewards = np.array(self.rewards)

        q0 = rewards[actions == 0].mean()
        q1 = rewards[actions == 1].mean()

        self.q_table = np.array([q0, q1])

* 학습

In [ ]:
env = Bandit()
player = Agent()

for _ in range(100) :
    player.act()
    player.reward = env.step(player.action)
    player.save_experience()
    player.learn()

    if (player.n_experience >= 20) and (np.array(player.rewards)[-20:].mean() > 9.5) :
        break

`-` 4x4 Grid World

* 클래스 선언

In [ ]:
class GridWorld :
    def __init__(self) :
        self.a2d = {
            0: np.array([0,1]),  # →
            1: np.array([0,-1]), # ←  
            2: np.array([1,0]),  # ↓
            3: np.array([-1,0])  # ↑
        }

        self.states_space = gym.spaces.MultiDiscrete([4, 4])
        self.state = np.array([0, 0])
        self.reward = None
        self.terminated = False

    def step(self, action) :
        self.state = self.state + self.a2d[action] ## 여기서 깊은복사 이슈 나는듯
        s1, s2 = self.state

        if (s1 == 3) and (s2 == 3) :
            self.reward = 100
            self.terminated = True
        
        elif self.state in self.states_space :
            self.reward = -1

        else :
            self.reward = -10
            self.terminated = True

        return self.state, self.reward, self.terminated

    def reset(self) :
        self.state = np.array([0, 0])
        self.terminated = False
        
        return self.state


class RandomAgent :
    def __init__(self) :
        self.state = np.array([0, 0])
        self.action = None
        self.reward = None
        self.next_state = None
        self.terminated = None

        self.states = collections.deque(maxlen = 500000)
        self.actions = collections.deque(maxlen = 500000)
        self.rewards = collections.deque(maxlen = 500000)
        self.next_states = collections.deque(maxlen = 500000)
        self.terminations = collections.deque(maxlen = 500000)

        self.action_space = gym.spaces.Discrete(4)
        self.n_experience = 0

    def act(self) :
        self.action = self.action_space.sample()

    def save_experience(self) :
        self.states.append(self.state)
        self.actions.append(self.action)
        self.rewards.append(self.reward)
        self.next_states.append(self.next_state)
        self.terminations.append(self.terminated)
        self.n_experience += 1

    def learn(self) :
        pass

* 학습

In [ ]:
player = RandomAgent()
env = GridWorld()
scores = [] 
score = 0 
#
for e in range(1,100000):
    #---에피소드시작---#
    while True:
        # step1 -- 액션선택
        player.act()
        # step2 -- 환경반응 
        player.next_state, player.reward, player.terminated = env.step(player.action)
        # step3 -- 경험기록 & 학습 
        player.save_experience()
        player.learn()
        # step4 --종료 조건 체크 & 후속 처리
        if env.terminated:
            score = score + player.reward
            scores.append(score)
            score = 0 
            player.state = env.reset() 
            break
        else: 
            score = score + player.reward
            scores.append(score)            
            player.state = player.next_state

> 실시간 학습이 아닌 전부 행동한 뒤에 학습하는 방식

`-` 평균을 이용한 `q_table`

In [ ]:
q_table = np.zeros([4, 4, 4])
count = np.zeros([4, 4, 4])

for (s1, s2), a, r in zip(player.states, player.actions, player.rewards) :
    q_table[s1, s2, a] += r
    count[s1, s2, a] += 1

count[count == 0] = 1e-6
q_table = q_table/count

`-` 부스팅 기법을 이용한 `q_table`

In [ ]:
q_table = np.zeros([4, 4, 4])

for (s1, s2), a, r in zip(player.states, player.actions, player.rewards) :
    qhat = q_table[s1, s2, a]
    q = r
    diff = qhat - q
    q_table[s1, s2, a] += 0.01*diff

`-` 감가율(Discount Rate)을 적용한 `q_table`

In [ ]:
def act(player,s1,s2):
    action = player.q_table[s1,s2,:].argmax()
    return action

In [ ]:
for t in range(1, 50) :
    s1, s2 = player.state
    action = act(s1, s2)
    player.next_state, player.reward, player.terminated = env.step(action)
    player.save_experience()
    player.learn()
    score += player.reward

`-` 개선된 Agent : Greedy Search

* 즉시 얻게되는 보상과 미래에 얻게되리라 기대되는 보상을 동일하게 취급할 수는 없음
* 모든 $(s, a)$에 대하여 $Q(s, a)$의 값은 아래와 같이 정의하는 게 합리적이다.

$$Q(s, a) = R(s, a) + \gamma \max_{a'} Q(s', a')$$

> 미래의 상황 중 가장 좋은 선택에 대해 감가율을 적용하여 더함.
> 
> **Note**
>
> 사실 좀 더 실용적으로는(=코딩친화적으로는) 아래의 수식을 쓰는게 좋다.
>
> $$Q(s,a) = \begin{cases}  R(s,a) & \text{if terminated} \\ R(s,a) + \gamma \max_{a'}Q(s',a') & \text{else} \end{cases}$$

In [ ]:
q_table = np.zeros((4,4,4))
memory = zip(player.states, player.actions, player.rewards, player.next_states, player.terminations)

for (s1,s2), a, r, (ss1, ss2), tmd in memory:
    qhat = q_table[s1,s2,a] # 내가 생각했던값
    
    if tmd :
        q = r # 실제값
    else :
        future = q_table[ss1, ss2, :].max()
        q = r + 0.95*future
        
    diff = q-qhat # 차이
    q_table[s1,s2,a] = q_table[s1,s2,a] + 0.01*diff# update

`-` `q_table`의 이용 및 시각화

In [ ]:
def show(states):
    fig = plt.Figure()
    ax = fig.subplots()
    ax.matshow(np.zeros([4,4]), cmap='bwr',alpha=0.0)
    sc = ax.scatter(0, 0, color='red', s=500)  
    ax.text(0, 0, 'start', ha='center', va='center')
    ax.text(3, 3, 'end', ha='center', va='center')
    # Adding grid lines to the plot
    ax.set_xticks(np.arange(-.5, 4, 1), minor=True)
    ax.set_yticks(np.arange(-.5, 4, 1), minor=True)
    ax.grid(which='minor', color='black', linestyle='-', linewidth=2)
    state_space = gym.spaces.MultiDiscrete([4,4])
    def update(t):
        if states[t] in state_space:
            s1,s2 = states[t]
            states[t] = [s2,s1]
            sc.set_offsets(states[t])
        else:
            s1,s2 = states[t]
            s1 = s1 + 0.5 if s1 < 0 else (s1 - 0.5 if s1 > 3 else s1)
            s2 = s2 + 0.5 if s2 < 0 else (s2 - 0.5 if s2 > 3 else s2)
            states[t] = [s2,s1]       
            sc.set_offsets(states[t])
    ani = FuncAnimation(fig,update,frames=len(states))
    display(IPython.display.HTML(ani.to_jshtml()))

In [ ]:
player.state = env.reset()
player.terminated = False
score = 0

for t in range(2**8) :
    ## step1 --- 액션 선택
    s1, s2 = player.state
    player.action = act(player, s1, s2)
    ## step2 --- 환경 반응
    player.next_state, player.reward, player.terminated = env.step(player.action)
    ## step3 --- 경헙기록 & 학습
    player.save_experience()
    player.learn()
    ## step4 --- 종료 조건 체크
    if env.terminated :
        score += player.reward
        player.state = env.reset()
        player.playtimes.append(t+1)
        break
    else :
        score += player.reward
        player.state = player.next_state

In [ ]:
states = np.concat([np.array([[0, 0]]), np.array(player.next_states)[-player.playtimes[-1]:]], axis = 0)
show(states)

### **B. `q_net`**

`-` 전략 : `q_table`에 대응하는 뭔가를 만들자

* 상태공간에서 가능한 행동이 한정되어있고, 매우 많지 않은 경우 모든 경우의 수를 조사 가능
* 아주 많은 경우에 대한 reward 값을 조사하는 것은 현실적으로 불가능

`-` 클래스 선언

In [1]:
class RandomAgent :
    def __init__(self) :
        self.state = None
        self.action = None
        self.reward = None
        self.next_state = None
        self.terminated = None
        # self.truncated = None ## 얜 필요 없음??

        self.states = collections.deque(maxlen = 5000)
        self.actions = collections.deque(maxlen = 5000)
        self.rewards = collections.deque(maxlen = 5000)
        self.next_states = collections.deque(maxlen = 5000)
        self.terminations = collections.deque(maxlen = 5000)
        # self.truncations = collections.deque(maxlen = 5000) ## state에 들어가 있어서 학습에 영향 X긴 한데, 그럼 terminations도...

        self.action_space = gym.spaces.Discrete(4) ## env에도 있으나, 간단하므로 만듦
        self.n_experiences = 0

    def act(self) :
        self.action = self.action_space.sample()

    def save_experience(self) :
        self.states.append(torch.tensor(self.state)) ## Pytorch 네트워크에 입력값으로 들어감
        self.actions.append(self.action)
        self.rewards.append(self.reward)
        self.next_states.append(torch.tensor(self.next_state))
        self.terminations.append(self.terminated)

        self.n_experiences += 1

    def learn(self) :
        pass


class Agent(RandomAgent) :
    def __init__(self) :
        super().__init__()

        self.q_net = torch.nn.Sequential(
            torch.nn.Linear(8, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 4)
        )
        self.optimizr = torch.optim.Adam(self.q_net.parameters())
        self.eps = 1.0

    def act(self) :
        if random.random() < self.eps :
            self.action = self.action_space.sample()
        else :
            state = torch.tensor(self.state)
            self.action = self.q_net(state).argmax().item()
            
    def learn(self) :
        if self.n_experiences > 64 :
            for epoc in range(1) :
                memory = list(zip(self.states, self.actions, self.rewards, self.next_states, self.terminations))
                mini_batch = random.sample(memory, 64)
            
                ## step 1-2
                loss = 0
                
                for s, a, r, ss, tmd in mini_batch :
                    q_hat = self.q_net(s)[a]
                    
                    if tmd :
                        q = r
                    else :
                        future = self.q_net(ss).max().data
                        q = r + 0.99*future
                        
                    loss += (q_hat-q)**2
                
                loss = loss/64
            
                ## step 3
                loss.backward()
            
                ## step 4
                self.optimizr.step()
                self.optimizr.zero_grad()

`-` 학습

In [ ]:
env = gym.make("LunarLander-v3", render_mode = "rgb_array")
player = Agent()
player.state, _ = env.reset()
score = 0
playtime = 0
scores = []
playtimes = []

for e in range(1, 2001) :
    ##-----에피소드 시작-----##
    while True :
        ## step 1
        player.act()
        ## step 2
        player.next_state, player.reward, player.terminated, player.truncated, _ = env.step(player.action)
        ## step 3
        player.save_experience()
        player.learn()
        ## step 4
        score += player.reward
        
        if player.terminated or player.truncated :
            scores.append(score)
            playtimes.append(playtime)
            score = 0
            playtime = 0
            player.state, _ = env.reset()
            break

        else :
            playtime += 1
            player.state = player.next_state

    ##-----에피소드 끝-----##
    player.eps = player.eps * 0.995
    if (e % 50) == 0 :
        print(
            f"에피소드: {e}\t",
            f"경험: {player.n_experiences}\t",
            f"점수(평균): {np.mean(scores[-100:]):.2f}\t",
            f"게임시간(평균): {np.mean(playtimes[-100:]):.2f}\t",
            f"돌발행동: {player.eps:.2f}\t",
        )

    if np.mean(scores[-100:]) > 200:
        print("--루나랜더 클리어(2025.06.14.)--")
        break

> 아주 오래 걸리고, 불안정함. 안정적인 학습을 위해서는 복잡한 테크닉을 사용해야 합니다...

`-` 문제 해결

In [ ]:
def show(imgs,jump=10):
    imgs = imgs[::jump]
    fig = plt.Figure()
    ax = fig.subplots()
    def update(i):
        ax.imshow(imgs[i])
    ani = FuncAnimation(fig,update,frames=len(imgs))
    display(IPython.display.HTML(ani.to_jshtml()))

In [ ]:
player.eps = 0

while True:
    player.act()
    player.next_state, player.reward, player.terminated, player.truncated, _ = env.step(player.action)
    imgs.append(env.render())
    if player.terminated or player.truncated:
        break
    else:
        player.state = player.next_state


show(imgs)

## **5. 용어 정리**

### **A. ${\bf X}$, ${\bf y}$**

`-` X, y를 지칭하는 이름

| 구분 | 용어                            | 설명                                                                                               |
|----------|-----------------------------------------------------|----------|
| X    | 설명변수                        | 종속변수(반응변수)를 설명하거나 예측하는 데 사용되는 변수로, 전통 통계 및 머신러닝에서의 입력 역할 |
|      | 독립변수 (Independent Variable) | 전통적인 통계학 및 회귀 분석 문맥에서 사용됨                                                       |
|      | 입력변수 (Input Variable)       | 머신러닝 모델에서 입력 데이터로 사용되며, 특히 신경망 구조 등에서 많이 쓰임                        |
|      | 특징 / 특성 (Feature)           | 머신러닝, 데이터마이닝, 딥러닝 등에서 데이터를 구성하는 속성 또는 설명 변수로 사용됨               |
|      | 예측 변수 (Predictor)           | 예측 모델 설계 시 독립변수를 지칭하는 용어로, 모델링/통계 분석 문맥에서 흔히 사용됨                |
|      | 공변량 (Covariate)              | 실험 디자인, 특히 임상연구나 사회과학 연구에서 제어 변수로 사용됨                                  |
| y    | 반응변수                        | 독립변수의 영향을 받는 결과 변수로, 모델링이나 인과 추론에서 핵심적인 대상                         |
|      | 종속변수 (Dependent Variable)   | 전통 통계학과 회귀분석에서 사용되며, 독립변수의 영향을 받는 변수로 정의됨                          |
|      | 출력변수 (Output Variable)      | 머신러닝 및 딥러닝에서 모델의 예측 결과로 출력되는 값으로 사용됨                                   |
|      | 타겟 / 정답 (Target / Label)    | 지도학습에서 모델이 학습해야 하는 실제 정답값을 의미하며, 분류/회귀 문제에 공통적으로 사용됨       |

`-` 더 다양함: <https://ko.wikipedia.org/wiki/독립변수와_종속변수>

### **B. 지도학습**

`-` 우리가 수업에서 다루는 데이터는 주로 아래와 같은 느낌이다.

1.  데이터는 $(X,y)$의 형태로 정리되어 있다.

2.  $y$는 우리가 관심이 있는 변수이다. 즉 우리는 $y$를 적절하게 추정하는
    것에 관심이 있다.

3.  $X$는 $y$를 추정하기 위해 필요한 정보이다.

|           $X$           |           $y$            |        비고        |      순서      |               예시               |
|:------------:|:--------------:|:----------:|:----------:|:------------------:|
|       기온(온도)        | 아이스 아메리카노 판매량 |        회귀        |    상관없음    | 날씨가 판매량에 미치는 영향 분석 |
|          스펙           |        합격 여부         |      로지스틱      |    상관없음    |     입사 지원자의 합격 예측      |
|         이미지          |         카테고리         | 합성곱신경망 (CNN) |    상관없음    |      개/고양이 이미지 구분       |
|    유저, 아이템 정보    |           평점           |     추천시스템     |    상관없음    |        넷플릭스 영화 추천        |
| 처음 $m$개의 단어(문장) |  이후 1개의 단어(문장)   |  순환신경망 (RNN)  | 순서 상관 있음 |   챗봇, 문장 생성, 언어 모델링   |
| 처음 $m$개의 단어(문장) |         카테고리         |  순환신경망 (RNN)  | 순서 상관 있음 |        영화리뷰 감정 분류        |

`-` 이러한 문제상황, 즉 $(X,y)$가 주어졌을때 $X \to y$를 추정하는 문제를
supervised learning 이라한다.

### **C. 모델이란?**

`-` 통계학에서 모델은 y와 x의 관계를 의미하며 오차항의 설계를 포함하는
개념이다. 이는 통계학이 “데이터 = 정보 + 오차”의 관점을 유지하기
때문이다. 따라서 통계학에서 모델링이란

$$y_i = net(x_i) + \epsilon_i$$

에서 (1) 적절한 함수 $net$를 선택하는 일 (2) 적절한 오차항 $\epsilon_i$
을 설계하는일 모두를 포함한다.

`-` 딥러닝 혹은 머신러닝에서 모델은 단순히

$$y_i \approx net(x_i)$$

를 의미하는 경우가 많다. 즉 “model=net”라고 생각해도 무방하다. 이 경우
“모델링”이란 단순히 적절한 $net$을 설계하는 것만을 의미할 경우가 많다.

`-` 그래서 생긴일

-   통계학교재 특징: 분류문제와 회귀문제를 엄밀하게 구분하지 않는다.
    사실 오차항만 다를뿐이지 크게보면 같은 회귀모형이라는 관점이다.
    그래서 일반화선형모형(GLM)이라는 용어를 쓴다.
-   머신러닝/딥러닝교재 특징: 회귀문제와 분류문제를 구분해서 설명한다.
    (표도 만듦) 이는 오차항에 대한 기술을 모호하게 하여 생기는 현상이다.

### **D. 학습이란?**

`-` 학습이란 주어진 자료 $(X,y)$를 잘 분석하여 $X$에서 $y$로 가는 어떠한
“규칙” 혹은 “원리”를 찾는 것이다.

-   학습이란 주어진 자료 $(X,y)$를 잘 분석하여 $X$에서 $y$로 가는 어떠한
    “맵핑”을 찾는 것이다.
-   학습이란 주어진 자료 $(X,y)$를 잘 분석하여 $X$에서 $y$로 가는 어떠한
    “함수”을 찾는 것이다. 즉 $y\approx f(X)$가 되도록 만드는 $f$를 잘
    찾는 것이다. (이 경우 “함수를 추정한다”라고 표현)
-   학습이란 주어진 자료 $(X,y)$를 잘 분석하여 $X$에서 $y$로 가는 어떠한
    “모델” 혹은 “모형”을 찾는 것이다. 즉 $y\approx model(X)$가 되도록
    만드는 $model$을 잘 찾는 것이다. (이 경우 “모형을 학습시킨다”라고
    표현)
-   **학습이란 주어진 자료 $(X,y)$를 잘 분석하여 $X$에서 $y$로 가는
    어떠한 “네트워크”을 찾는 것이다. 즉 $y\approx net(X)$가 되도록
    만드는 $net$을 잘 찾는 것이다. (이 경우 “네트워크를 학습시킨다”라고
    표현)**

`-` prediction이란 학습과정에서 찾은 “규칙” 혹은 “원리”를 $X$에 적용하여
$\hat{y}$을 구하는 과정이다. 학습과정에서 찾은 규칙 혹은 원리는
$f$,$model$,$net$ 으로 생각가능한데 이에 따르면 아래가 성립한다.

-   $\hat{y} = f(X)$
-   $\hat{y} = model(X)$
-   $\hat{y} = net(X)$

### **E. $\hat{y}$를 부르는 다양한 이름**

`-` $\hat{y}$는 $X$가 주어진 자료에 있는 값인지 아니면 새로운 값 인지에
따라 지칭하는 이름이 미묘하게 다르다.

1.  $X \in data$: $\hat{y}=net(X)$ 는 predicted value, fitted value 라고
    부른다.

2.  $X \notin data$: $\hat{y}=net(X)$ 는 predicted value, predicted
    value with new data 라고 부른다.

### **F. 다양한 코드들**

`-` 파이썬 코드..

``` python
#Python
predictor.fit(X,y) # autogluon 에서 "학습"을 의미하는 과정
model.fit(X,y) # sklearn 에서 "학습"을 의미하는 과정
trainer.train() # huggingface 에서 "학습"을 의미하는 과정
trainer.predict(dataset) # huggingface 에서 "예측"을 의미하는 과정
model.fit(x, y, batch_size=32, epochs=10) # keras에서 "학습"을 의미하는 과정
model.predict(test_img) # keras에서 "예측"을 의미하는 과정 
```

`-` R 코드..

``` r
# R
ols <- lm(y~x) # 선형회귀분석에서 학습을 의미하는 함수
ols$fitted.values # 선형회귀분석에서 yhat을 출력 
predict(ols, newdata=test) # 선형회귀분석에서 test에 대한 예측값을 출력하는 함수
ols$coef # 선형회귀분석에서 weight를 확인하는 방법
```

### **G. 신경망관련용어**

-   ANN: 인공신경망
-   MLP: 다층퍼셉트론 (레이어가 여러개 있어요)
-   DNN: 깊은신경망, 심층신경망 - 얼마나 깊어야...? 대충 세 개 이상 정도...
-   CNN: 합성곱신경망 - 컨볼루션 커널 쓰면 되니까 명확함
-   RNN: 순환신경망 - 순환 구조가 있어야 함

`# 예시1` – MLP, DNN

``` python
net = torch.nn.Sequential(
    torch.nn.Linear(in_features=1,out_features=2),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features=2,out_features=2),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features=2,out_features=1),    
    torch.nn.Sigmoid()
)
```

-   ANN: O
-   MLP: O
-   DNN: O
-   CNN: X (합성곱레이어가 없으므로)
-   RNN: X (순환구조가 없으므로)

`#`

`# 예시2` – MLP, Shallow Network

``` python
net = torch.nn.Sequential(
    torch.nn.Linear(in_features=1,out_features=2),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features=2,out_features=1),
    torch.nn.Sigmoid()
)
```

-   ANN: O
-   MLP: O
-   DNN: X (깊은 신경망으로 생각하려면 더 많은 레이어가 필요함. 합의된
    기준은 히든레이어 2장이상, 이걸 설명하기 위해서 얕은 신경망이란
    용어도 씀)
-   CNN: X (합성곱레이어가 없으므로)
-   RNN: X (순환구조가 없으므로)

`#`

`# 예시3` – MLP, DNN, Wide NN

``` python
net = torch.nn.Sequential(
    torch.nn.Linear(in_features=1,out_features=1048576),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features=1048576,out_features=1048576),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features=1048576,out_features=1),
    torch.nn.Sigmoid(),    
)
```

-   ANN: O
-   MLP: O
-   DNN: O (깊긴한데 이정도면 모양이 깊다기 보다는 넓은 신경망임, 그래서
    어떤 연구에서는 이걸 넓은 신경망이라 부르기도 함)
-   CNN: X (합성곱레이어가 없으므로)
-   RNN: X (순환구조가 없으므로)

`# 예시4` – CNN

``` python
net = torch.nn.Sequential(
    # Layer1
    torch.nn.Conv2d(1, 64, kernel_size=4, stride=2, padding=1, bias=False),
    torch.nn.LeakyReLU(0.2),
    # Layer2
    torch.nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
    torch.nn.BatchNorm2d(128),
    torch.nn.LeakyReLU(0.2),
    # Layer3
    torch.nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
    torch.nn.BatchNorm2d(256),
    torch.nn.LeakyReLU(0.2),
    # Layer4
    torch.nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias=False),
    torch.nn.BatchNorm2d(512),
    torch.nn.LeakyReLU(0.2),
    # Layer5
    torch.nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=0, bias=False),
    torch.nn.Sigmoid(),
    torch.nn.Flatten()
)
```

-   ANN: O
-   MLP: X (합성곱연결이 포함되어있으므로, MLP가 아님, 완전연결만
    포함해야 MLP임)  
-   DNN: O (그냥 레이어가 3층 이상이면 무지성으로 DNN임)
-   CNN: O (합성곱레이어를 포함하고 있으므로)
-   RNN: X (순환구조가 없으므로)

`#`

`# 예시5` – CNN

``` python
net = torch.nn.Sequential(
    torch.nn.Conv2d(1,16,(5,5)),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d((2,2)),
    torch.nn.Flatten(),
    torch.nn.Linear(2304,1),
    torch.nn.Sigmoid()
)
```

-   ANN: O
-   MLP: X
-   DNN: X? (히든레이어가 1장이므로..)
-   CNN: O (합성곱레이어를 포함하고 있으므로)
-   RNN: X (순환구조가 없으므로)

> 근데 대부분의 문서에서는 CNN, RNN은 DNN의 한 종류로 설명하고
> 있어서요.. 이런 네트워크에서는 개념충돌이 옵니다.

`#`

`# 예시6` – RNN

``` python
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = torch.nn.RNN(4,2)
        self.linr = torch.nn.Linear(2,2) 
    def forward(self,X):
        h,_ = self.rnn(X) 
        netout = self.linr(h)
        return netout 
net = Net()     
```

-   ANN: O
-   MLP: X
-   DNN: X? (히든레이어가 1장이므로..)
-   CNN: X (합성곱레이어가 없으므로)
-   RNN: O

> 이것도 비슷한 개념충돌